In [1]:
!pip install backoff
! pip install SPARQLWrapper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 3.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.7 MB/s eta 0:00:00


In [1]:
import json
import random
import os
import pandas as pd
from tqdm.asyncio import tqdm
import re
import aiohttp
import asyncio
import backoff
import nest_asyncio
import time
from SPARQLWrapper import SPARQLWrapper, JSON
from requests import get
import numpy as np
import requests

# Round1

In [3]:
####################
# READ THE JSON
#####################

json_file_path = "./data/Round1_T2D_f3_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R1_sorted_mentions = json.load(file)

In [4]:
# SPLIT OVER THE QUARTILES

n = len(R1_sorted_mentions)
q1_idx = n // 4
q2_idx = n // 2
q3_idx = 3 * n // 4

# Step 3: Split the list into quartiles
q1 = R1_sorted_mentions[:q1_idx]
q2 = R1_sorted_mentions[q1_idx:q2_idx]
q3 = R1_sorted_mentions[q2_idx:q3_idx]
q4 = R1_sorted_mentions[q3_idx:]

sample_size = 1000
R1_sample_keys = []
R1_sample_keys = R1_sample_keys + random.sample(q1, sample_size)
R1_sample_keys = R1_sample_keys + random.sample(q2, sample_size)
R1_sample_keys = R1_sample_keys + random.sample(q3, sample_size)
R1_sample_keys = R1_sample_keys + random.sample(q4, sample_size)

q_ids = {item[1]['name']: item[1]['id'] for item in R1_sample_keys}

In [5]:
# find the mention in the table
tables = "./data/Dataset/Dataset/Round1_T2D/tables/"
cta_file = './data/Dataset/Dataset/Round1_T2D/gt/CTA_Round1_gt.csv'
os.listdir(tables)

mapping = {
    "LOC": [
        "Place", "PopulatedPlace", "City", "Country", "Region", "Mountain", "Island", "Lake", "River",
        "Park", "Building", "HistoricPlace", "Monument", "Bridge", "Road", "Airport"
    ],
    "PERS": [
        "Person", "Artist", "Athlete", "Politician", "Scientist", "Writer", "Actor", "Musician", "MilitaryPerson",
        "Religious", "Royalty", "Criminal"
    ],
    "ORG": [
        "Organisation", "Company", "EducationalInstitution", "PoliticalParty", "SportsTeam", "Non-ProfitOrganisation",
        "GovernmentAgency", "ReligiousOrganisation", "Band", "Library", "Museum", "Hospital", "University", "TradeUnion"
    ]
}

# Create reverse mapping
reverse_mapping = {v: k for k, values in mapping.items() for v in values}

# Define function to map df[2] values to their categories
def map_class_to_category(class_name):
    return reverse_mapping.get(class_name, "OTHERS")

# Apply the function and create the 'key' column
cta_keys = {}
df = pd.read_csv(cta_file, header=None)
type = df[2].astype(str).str.split('/').str[-1]
df["category"] = type.apply(map_class_to_category)
cta_keys["key"] = (df[0] + " " + df[1].astype('str'), df["category"])

key_to_cell = {}
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            key = f"{table_name} {col}"
            if key in set(cta_keys["key"][0].values):
                tmp_index = cta_keys["key"][0].values.tolist().index(key)
                tmp_value = cta_keys["key"][1].iloc[tmp_index]
                key_to_cell[key] = tmp_value

100%|██████████| 64/64 [00:01<00:00, 37.47it/s]


In [6]:
def get_keys_from_value(d, value):
    keys = [key for key, val in d.items() if val == value]
    return keys[0]

In [7]:
cea_file = './data/Dataset/Dataset/Round1_T2D/gt/CEA_Round1_gt_WD.csv'
mentions = {}
chunk_size = 1000
column_names = ["table_name", "row", "col", "url"] 

total_rows = sum(1 for line in open(cea_file)) - 1  # Exclude header
total_iterations = (total_rows + chunk_size - 1) // chunk_size  # Ceiling division to include last chunk

count = 0
for chunk_cea in tqdm(pd.read_csv(cea_file, chunksize=chunk_size), total=total_iterations):
    chunk_cea.columns = column_names
    for _, row in chunk_cea.iterrows():
        key = f"{row['table_name']} {row['col']}"
        if key in key_to_cell.keys() and row["url"] in q_ids.values():
            count += 1
            data = key_to_cell[key]
            mentions[get_keys_from_value(q_ids, row["url"])] = (row["url"], data)

print("Processing complete.")

100%|██████████| 9/9 [00:02<00:00,  3.45it/s]

Processing complete.


In [8]:
url = 'https://lamapi.hel.sintef.cloud/lookup/entity-retrieval'

# Backoff decorator for handling retries with exponential backoff
@backoff.on_exception(
    backoff.expo, 
    (aiohttp.ClientError, aiohttp.http_exceptions.HttpProcessingError, asyncio.TimeoutError), 
    max_tries=5, 
    max_time=300
)
async def fetch(session, url, params, headers, semaphore):
    async with semaphore:
        async with session.get(url, params=params, headers=headers, timeout=30) as response:
            try:
                response.raise_for_status()  # Raises an exception for 4XX/5XX status codes
                return await response.json()
            except Exception as e:
                return []
async def process_item(session, name, value, url, headers, semaphore, pbar):
    params = {
        'name': name,
        'token': 'lamapi_demo_2023',
        'kg': 'wikidata',
        'limit': 1000,
        'query': f'''
            {{
                "query": {{
                    "bool": {{
                        "must": [
                            {{
                                "match": {{
                                    "name": {{
                                        "query": "{name}",
                                        "boost": 2.0
                                    }}
                                }}
                            }}
                        ],
                        "should": [
                            {{
                                "term": {{
                                    "NERtype": "{value[1]}"
                                }}
                            }}
                        ]
                    }}
                }}
            }}
            ''',
        'sort': [
            f'''{{"popularity": {{"order": "desc"}}}}'''
        ]
    }


    try:
        data = await fetch(session, url, params, headers, semaphore)
    except ClientResponseError as e:
        if e.status == 404:
            print(f"404 Error: Resource not found for '{name}'")
            pbar.update(1)  # No need to await here
            return 0, 0
        else:
            raise  # Re-raise the exception for other status codes

    num_result = len(data) if data else 0

    if data:
        for item in data:
            GT_id_match = re.search(r'Q(\d+)$', value[0])
            if GT_id_match:
                GT_id = GT_id_match[0]
                if GT_id == item.get('id'):
                    pbar.update(1)  # No need to await here
                    pos_score = item.get('pos_score', 0)
                    if pos_score:
                        mrr_increment = (num_result - (pos_score * num_result)) / num_result
                    else:
                        mrr_increment = 1 / num_result  # Assume worst case for MRR if pos_score is 0
                    return mrr_increment, 1

        #print(f"{name} NOT FOUND-->t{item}")

    return 0, 0

async def main(mentions, url, pbar):
    string_name_list = mentions
    headers = {'accept': 'application/json'}
    semaphore = asyncio.Semaphore(50)  # Limit to 50 concurrent requests
    m_mrr = 0
    cont_el = 0
    async with aiohttp.ClientSession() as session:
        tasks = []
        for name, type in string_name_list.items():
            tasks.append(process_item(session, name, type, url, headers, semaphore, pbar))
        
        results = await asyncio.gather(*tasks)
        
        for (mrr_increment, count), (name, url_id) in zip(results, string_name_list.items()):
            if mrr_increment == 0 and count == 0:
                params = {
                    'name': name,
                    'token': 'lamapi_demo_2023',
                    'kg': 'wikidata',
                    'limit': 1000,
                    'query':  f'''{{"query": {{"bool": {{"must": [{{"match": {{"name": {{"query": "{name}", "boost": 2.0, "fuzziness": "AUTO"}}}}}}]}}}}}}''',
                    'sort': [
                        f'''{{"popularity": {{"order": "desc"}}}}'''
                    ]
                }
                id = re.search(r'Q(\d+)$', url_id[0])[0]
                
                response = requests.get(url, params)
                if response.status_code == 200:
                    data = response.json()
                    #print("after call")
                    num_result = len(data) if data else 0
                    if data:
                        for item in data:
                            if id == item.get('id'):
                                pbar.update(1)  # No need to await here
                                pos_score = item.get('pos_score', 0)
                                if pos_score:
                                    mrr_increment = (num_result - (pos_score * num_result)) / num_result
                                else:
                                    mrr_increment = 1 / num_result  # Assume worst case for MRR if pos_score is 0
                            
            m_mrr += mrr_increment
            cont_el += count

        pbar.close()  # No need to await here

    print(f"Coverage of R1: {cont_el / len(mentions)}")
    print(f"Measure Reciprocal Rank of R1: {m_mrr / len(mentions)}")

# Check if there's already a running event loop
if __name__ == "__main__":
    nest_asyncio.apply()  # Apply nest_asyncio
    try:
        pbar = tqdm(total=len(mentions))
        asyncio.run(main(mentions, url, pbar))
    except RuntimeError:  # For environments like Jupyter
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main(mentions, url, pbar))


100%|█████████▉| 3856/3859 [14:00<00:00,  4.59it/s]

Coverage of R1: 0.9836745270795543
Measure Reciprocal Rank of R1: 0.9616820419797453


## Coverage with the filtering
Coverage of R1: 0.9836745270795543

Measure Reciprocal Rank of R1: 0.96168204197974533

# Round3

In [9]:
####################
# READ THE JSON
#####################

json_file_path = "./data/Round3_2019_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R3_sorted_mentions = json.load(file)

In [10]:
# SPLIT OVER THE QUARTILES

n = len(R3_sorted_mentions)
q1_idx = n // 4
q2_idx = n // 2
q3_idx = 3 * n // 4

# Step 3: Split the list into quartiles
q1 = R3_sorted_mentions[:q1_idx]
q2 = R3_sorted_mentions[q1_idx:q2_idx]
q3 = R3_sorted_mentions[q2_idx:q3_idx]
q4 = R3_sorted_mentions[q3_idx:]


sample_size = 1000 
R3_sample_keys = []
R3_sample_keys = R3_sample_keys + random.sample(q1, sample_size)
R3_sample_keys = R3_sample_keys + random.sample(q2, sample_size)
R3_sample_keys = R3_sample_keys + random.sample(q3, sample_size)
R3_sample_keys = R3_sample_keys + random.sample(q4, sample_size)

q_ids = {item[1]['name']: item[1]['id'] for item in R3_sample_keys}

In [11]:
# find the mention in the table
tables = "./data/Dataset/Dataset/Round3_2019/tables/"
cta_file = './data/Dataset/Dataset/Round3_2019/gt/CTA_Round3_gt.csv'
os.listdir(tables)


# Apply the function and create the 'key' column
cta_keys = {}
df = pd.read_csv(cta_file, header=None)
category_list = []

for row_idx in range(df.shape[0]):
    col_idx = 2
    while True:
        try:
            if pd.isna(df.iloc[row_idx,col_idx]):
                category_list.append("OTHERS")
                break
            urls = df.iloc[row_idx,col_idx].split(' ')
        except IndexError as e:
            category_list.append("OTHERS")
            break
        
        #print(f"{df.iloc[row_idx,0]}->{cell_urls} @ {row_idx},{col_idx}")
        find = False
        for url in urls:
            type = url.split('/')[-1]            
            if type == "Person":
                category_list.append("PER")
                find = True
                break
            elif type == "Location":
                category_list.append("LOC")
                find = True
                break
            elif type == "Organisation":
                category_list.append("ORG")
                find = True
                break
        if find:
            break
        
        col_idx += 1



df["category"] = category_list
cta_keys = {}
cta_keys["key"] = (df[0] + " " + df[1].astype('str'), df["category"])

key_to_cell = {}
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            key = f"{table_name} {col}"
            if key in set(cta_keys["key"][0].values):
                tmp_index = cta_keys["key"][0].values.tolist().index(key)
                tmp_value = cta_keys["key"][1].iloc[tmp_index]
                key_to_cell[key] = tmp_value

100%|██████████| 2161/2161 [06:03<00:00,  5.94it/s]


In [12]:
def get_keys_from_value(d, value):
    keys = [key for key, val in d.items() if val == value]
    return keys[0]

In [13]:
cea_file = './data/Dataset/Dataset/Round3_2019/gt/CEA_Round3_gt_WD.csv'
mentions = {}
chunk_size = 1000
column_names = ["table_name", "row", "col", "url"] 

total_rows = sum(1 for line in open(cea_file)) - 1  # Exclude header
total_iterations = (total_rows + chunk_size - 1) // chunk_size  # Ceiling division to include last chunk

for chunk_cea in tqdm(pd.read_csv(cea_file, chunksize=chunk_size), total=total_iterations):
    chunk_cea.columns = column_names
    for _, row in chunk_cea.iterrows():
        key = f"{row['table_name']} {row['col']}"
        if key in key_to_cell.keys() and row["url"] in q_ids.values():
            data = key_to_cell[key]
            mentions[get_keys_from_value(q_ids, row["url"])] = (row["url"], data)

print("Processing complete.")

100%|██████████| 391/391 [04:35<00:00,  1.42it/s]

Processing complete.


In [14]:
url = 'https://lamapi.hel.sintef.cloud/lookup/entity-retrieval'

# Backoff decorator for handling retries with exponential backoff
@backoff.on_exception(
    backoff.expo, 
    (aiohttp.ClientError, aiohttp.http_exceptions.HttpProcessingError, asyncio.TimeoutError), 
    max_tries=5, 
    max_time=300
)
async def fetch(session, url, params, headers, semaphore):
    async with semaphore:
        async with session.get(url, params=params, headers=headers, timeout=30) as response:
            try:
                response.raise_for_status()  # Raises an exception for 4XX/5XX status codes
                return await response.json()
            except Exception as e:
                return []
async def process_item(session, name, value, url, headers, semaphore, pbar):
    params = {
        'name': name,
        'token': 'lamapi_demo_2023',
        'kg': 'wikidata',
        'limit': 1000,
        'query': f'''
            {{
                "query": {{
                    "bool": {{
                        "must": [
                            {{
                                "match": {{
                                    "name": {{
                                        "query": "{name}",
                                        "boost": 2.0
                                    }}
                                }}
                            }}
                        ],
                        "should": [
                            {{
                                "term": {{
                                    "NERtype": "{value[1]}"
                                }}
                            }}
                        ]
                    }}
                }}
            }}
            ''',
        'sort': [
            f'''{{"popularity": {{"order": "desc"}}}}'''
        ]
    }


    try:
        data = await fetch(session, url, params, headers, semaphore)
    except ClientResponseError as e:
        if e.status == 404:
            print(f"404 Error: Resource not found for '{name}'")
            pbar.update(1)  # No need to await here
            return 0, 0
        else:
            raise  # Re-raise the exception for other status codes

    num_result = len(data) if data else 0

    if data:
        for item in data:
            GT_id_match = re.search(r'Q(\d+)$', value[0])
            if GT_id_match:
                GT_id = GT_id_match[0]
                if GT_id == item.get('id'):
                    pbar.update(1)  # No need to await here
                    pos_score = item.get('pos_score', 0)
                    if pos_score:
                        mrr_increment = (num_result - (pos_score * num_result)) / num_result
                    else:
                        mrr_increment = 1 / num_result  # Assume worst case for MRR if pos_score is 0
                    return mrr_increment, 1

        #print(f"{name} NOT FOUND-->t{item}")

    return 0, 0

async def main(mentions, url, pbar):
    string_name_list = mentions
    headers = {'accept': 'application/json'}
    semaphore = asyncio.Semaphore(50)  # Limit to 50 concurrent requests
    m_mrr = 0
    cont_el = 0
    async with aiohttp.ClientSession() as session:
        tasks = []
        for name, type in string_name_list.items():
            tasks.append(process_item(session, name, type, url, headers, semaphore, pbar))
        
        results = await asyncio.gather(*tasks)
        
        for (mrr_increment, count), (name, url_id) in zip(results, string_name_list.items()):
            if mrr_increment == 0 and count == 0:
                params = {
                    'name': name,
                    'token': 'lamapi_demo_2023',
                    'kg': 'wikidata',
                    'limit': 1000,
                    'query':  f'''{{"query": {{"bool": {{"must": [{{"match": {{"name": {{"query": "{name}", "boost": 2.0, "fuzziness": "AUTO"}}}}}}]}}}}}}''',
                    'sort': [
                        f'''{{"popularity": {{"order": "desc"}}}}'''
                    ]
                }
                id = re.search(r'Q(\d+)$', url_id[0])[0]
                
                response = requests.get(url, params)
                if response.status_code == 200:
                    data = response.json()
                    #print("after call")
                    num_result = len(data) if data else 0
                    if data:
                        for item in data:
                            if id == item.get('id'):
                                pbar.update(1)  # No need to await here
                                pos_score = item.get('pos_score', 0)
                                if pos_score:
                                    mrr_increment = (num_result - (pos_score * num_result)) / num_result
                                else:
                                    mrr_increment = 1 / num_result  # Assume worst case for MRR if pos_score is 0
                            
            m_mrr += mrr_increment
            cont_el += count

        pbar.close()  # No need to await here

    print(f"Coverage of R3: {cont_el / len(mentions)}")
    print(f"Measure Reciprocal Rank of R3: {m_mrr / len(mentions)}")

# Check if there's already a running event loop
if __name__ == "__main__":
    nest_asyncio.apply()  # Apply nest_asyncio
    try:
        pbar = tqdm(total=len(mentions))
        asyncio.run(main(mentions, url, pbar))
    except RuntimeError:  # For environments like Jupyter
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main(mentions, url, pbar))


4230it [16:01,  4.40it/s]                          

Coverage of R3: 0.9634817408704353
Measure Reciprocal Rank of R3: 0.9472711355677341


## Coverage with the filteringCoverage of R3: 0.9634817408704353

Measure Reciprocal Rank of R3: 0.94727113556773416

# 2T_Round4

In [15]:
####################
# READ THE JSON
#####################

json_file_path = "./data/2T_Round4_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R4_2T_sorted_mentions = json.load(file)

In [16]:
## Sample extraction
# SPLIT OVER THE QUARTILES

n = len(R4_2T_sorted_mentions)
q1_idx = n // 4
q2_idx = n // 2
q3_idx = 3 * n // 4

# Step 3: Split the list into quartiles
q1 = R4_2T_sorted_mentions[:q1_idx]
q2 = R4_2T_sorted_mentions[q1_idx:q2_idx]
q3 = R4_2T_sorted_mentions[q2_idx:q3_idx]
q4 = R4_2T_sorted_mentions[q3_idx:]

sample_size = 1000
R4_2T_sample_keys = []
R4_2T_sample_keys = R4_2T_sample_keys + random.sample(q1, sample_size)
R4_2T_sample_keys = R4_2T_sample_keys + random.sample(q2, sample_size)
R4_2T_sample_keys = R4_2T_sample_keys + random.sample(q3, sample_size)
R4_2T_sample_keys = R4_2T_sample_keys + random.sample(q4, sample_size)

q_ids = {item[1]['name']: item[1]['id'] for item in R4_2T_sample_keys}

In [ ]:
def get_wikidata_item_tree_item_idsSPARQL(root_items, forward_properties=None, backward_properties=None):
    """Return ids of WikiData items, which are in the tree spanned by the given root items and claims relating them
        to other items.

    :param root_items: iterable[int] One or multiple item entities that are the root elements of the tree
    :param forward_properties: iterable[int] | None property-claims to follow forward; that is, if root item R has
        a claim P:I, and P is in the list, the search will branch recursively to item I as well.
    :param backward_properties: iterable[int] | None property-claims to follow in reverse; that is, if (for a root
        item R) an item I has a claim P:R, and P is in the list, the search will branch recursively to item I as well.
    :return: iterable[int]: List with ids of WikiData items in the tree
    """

    query = '''PREFIX wikibase: <http://wikiba.se/ontology#>
            PREFIX wd: <http://www.wikidata.org/entity/>
            PREFIX wdt: <http://www.wikidata.org/prop/direct/>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>'''
    if forward_properties:
        query +='''SELECT ?WD_id WHERE {
                  ?tree0 (wdt:P%s)* ?WD_id .
                  BIND (wd:%s AS ?tree0)
                  }'''%( ','.join(map(str, forward_properties)),','.join(map(str, root_items)))
    elif backward_properties:
        query+='''SELECT ?WD_id WHERE {
                    ?WD_id (wdt:P%s)* wd:Q%s .
                    }'''%(','.join(map(str, backward_properties)), ','.join(map(str, root_items)))
    #print(query)

    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    data = get(url, params={'query': query, 'format': 'json'}).json()
    
    ids = []
    for item in data['results']['bindings']:
        this_id=item["WD_id"]["value"].split("/")[-1].lstrip("Q")
        #print(item)
        try:
            this_id = int(this_id)
            ids.append(this_id)
            #print(this_id)
        except ValueError:
            #print("exception")
            continue
    return ids


try:
    organization_subclass = get_wikidata_item_tree_item_idsSPARQL([43229], backward_properties=[279])
    #print(len(organization_subclass))
except json.decoder.JSONDecodeError:
    pass

try:
    country_subclass = get_wikidata_item_tree_item_idsSPARQL([6256], backward_properties=[279])
except json.decoder.JSONDecodeError:
    country_subclass = set()
    pass

try:
    city_subclass = get_wikidata_item_tree_item_idsSPARQL([515], backward_properties=[279])
except json.decoder.JSONDecodeError:
    city_subclass = set()
    pass

try:
    capitals_subclass = get_wikidata_item_tree_item_idsSPARQL([5119], backward_properties=[279])
except json.decoder.JSONDecodeError:
    capitals_subclass = set()
    pass

try:
    admTerr_subclass = get_wikidata_item_tree_item_idsSPARQL([15916867], backward_properties=[279])
except json.decoder.JSONDecodeError:
    admTerr_subclass = set()
    pass

try:
    family_subclass = get_wikidata_item_tree_item_idsSPARQL([17350442], backward_properties=[279])
except json.decoder.JSONDecodeError:
    family_subclass = set()
    pass

try:
    sportLeague_subclass = get_wikidata_item_tree_item_idsSPARQL([623109], backward_properties=[279])
except json.decoder.JSONDecodeError:
    sportLeague_subclass = set()
    pass

try:
    venue_subclass = get_wikidata_item_tree_item_idsSPARQL([8436], backward_properties=[279])
except json.decoder.JSONDecodeError:
    venue_subclass = set()
    pass
    
try:
    organization_subclass = list(set(organization_subclass) - set(country_subclass) - set(city_subclass) - set(capitals_subclass) - set(admTerr_subclass) - set(family_subclass) - set(sportLeague_subclass) - set(venue_subclass))
    #print(len(organization_subclass))
except json.decoder.JSONDecodeError:
    pass


try:
    geolocation_subclass = get_wikidata_item_tree_item_idsSPARQL([2221906], backward_properties=[279])
    #print(len(geolocation_subclass))
except json.decoder.JSONDecodeError:
    pass

try:
    food_subclass = get_wikidata_item_tree_item_idsSPARQL([2095], backward_properties=[279])
except json.decoder.JSONDecodeError:
    food_subclass = set()
    pass

try:
    edInst_subclass = get_wikidata_item_tree_item_idsSPARQL([2385804], backward_properties=[279])
except json.decoder.JSONDecodeError:
    edInst_subclass = set()
    pass

try:
    govAgency_subclass = get_wikidata_item_tree_item_idsSPARQL([327333], backward_properties=[279])
except json.decoder.JSONDecodeError:
    govAgency_subclass = set()
    pass

try:
    intOrg_subclass = get_wikidata_item_tree_item_idsSPARQL([484652], backward_properties=[279])
except json.decoder.JSONDecodeError:
    intOrg_subclass = set()
    pass

try:
    timeZone_subclass = get_wikidata_item_tree_item_idsSPARQL([12143], backward_properties=[279])
except json.decoder.JSONDecodeError:
    timeZone_subclass = set()
    pass
   
try:
    geolocation_subclass = list(set(geolocation_subclass) - set(food_subclass) - set(edInst_subclass) - set(govAgency_subclass) - set(intOrg_subclass) - set(timeZone_subclass))
    #print(len(geolocation_subclass))
except json.decoder.JSONDecodeError:
    pass

try:
    human_subclass = get_wikidata_item_tree_item_idsSPARQL([5], backward_properties=[279])
    print(human_subclass)
except json.decoder.JSONDecodeError:
    human_subclass = set()
    pass

In [36]:
import os
import pandas as pd
from tqdm import tqdm
import logging

tables_path = "./data/Dataset/Dataset/2T_Round4/tables/"
cea_file = './data/Dataset/Dataset/2T_Round4/gt/cea.csv'
os.listdir(tables)
# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Read the cea_file and create a key-value dictionary
df = pd.read_csv(cea_file, header=None)
df["key"] = df[0] + " " + df[1].astype(str) + " " + df[2].astype(str)
cea_values_dict = dict(zip(df["key"].values, df[3].values))
cea_keys_set = set(df["key"].values)

# Function to process a single table file
def process_table_file(table_file):
    try:
        table_name = os.path.splitext(os.path.basename(table_file))[0]
        df = pd.read_csv(table_file)
        local_key_to_cell = {}
        
        for row in range(df.shape[0]):
            for col in range(df.shape[1]):
                key = f"{table_name} {row+1} {col}"
                if key in cea_keys_set:
                    cell_value = df.iloc[row, col]
                    local_key_to_cell[key] = (cell_value, cea_values_dict[key])
                    break  # Exit inner loop early as only one match per row/col is needed
        
        return local_key_to_cell
    except Exception as e:
        logging.error(f"Error processing {table_file}: {e}")
        return {}

# List of table files
table_files = [os.path.join(tables_path, table) for table in os.listdir(tables_path)]

# Process tables sequentially
key_to_cell = {}
for table_file in tqdm(table_files, desc="Processing tables"):
    local_key_to_cell = process_table_file(table_file)
    key_to_cell.update(local_key_to_cell)


Processing tables: 100%|██████████| 180/180 [00:07<00:00, 24.68it/s]


In [21]:
tables = "./data/Dataset/Dataset/2T_Round4/tables/"
cea_file = './data/Dataset/Dataset/2T_Round4/gt/cea.csv'
cta_file = './data/Dataset/Dataset/2T_Round4/gt/cta.csv'
os.listdir(tables)

def get_item_root(id_list):     
    id_to_root_class = {}
    for el in id_list:
        inst_item = int(re.search(r'(\d+)$', el)[0])
        if inst_item in geolocation_subclass:
            #id_to_root_class[el] = "LOC"
            return "LOC"
        elif inst_item in organization_subclass:
            #id_to_root_class[el] = "ORG"
            return "ORG"
        elif inst_item in human_subclass:
            #id_to_root_class[el] = "PERS"
            return "PERS"      
    
    return "OTHERS"

# Apply the function and create the 'key' column
root_classes = []
df = pd.read_csv(cta_file, header=None)
root_categories = []
for urls in df[2]:
    tmp = [url.split('/')[-1] for url in urls.split(" ")]
    root_categories.append(get_item_root(tmp))




df["category"] = root_categories
cta_keys = {}
cta_keys["key"] = (df[0] + " " + df[1].astype('str'), df["category"])

ner_type = {}
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            key = f"{table_name} {col}"
            if key in set(cta_keys["key"][0].values):
                tmp_index = cta_keys["key"][0].values.tolist().index(key)
                tmp_value = cta_keys["key"][1].iloc[tmp_index]
                ner_type[key] = tmp_value

100%|██████████| 180/180 [00:31<00:00,  5.64it/s]


In [22]:
def get_query(name, value):
    if value is not None:
         params = {
            'name': name,
            'token': 'lamapi_demo_2023',
            'kg': 'wikidata',
            'limit': 1000,
            'query': f'''
                {{
                    "query": {{
                        "bool": {{
                            "must": [
                                {{
                                    "match": {{
                                        "name": {{
                                            "query": "{name}",
                                            "boost": 2.0
                                        }}
                                    }}
                                }}
                            ],
                            "should": [
                                {{
                                    "term": {{
                                        "NERtype": "{value[1]}"
                                    }}
                                }}
                            ]
                        }}
                    }}
                }}
                ''',
            'sort': [
                f'''{{"popularity": {{"order": "desc"}}}}'''
            ]
        }
    else:
        params = {
            'name': name,
            'token': 'lamapi_demo_2023',
            'kg': 'wikidata',
            'limit': 1000,
            'query': f'{{"query": {{"bool": {{"must": [{{"match": {{"name": {{"query": "{name}", "boost": 2.0}}}}}}]}}}}}}',
            'sort': [
                f'''{{"popularity": {{"order": "desc"}}}}'''
            ]
        }
    return params


queries = []
for key in tqdm(key_to_cell):
    id_table, _, id_col = key.split(" ")
    name = key_to_cell[key][0]
    q_id = key_to_cell[key][1]
    new_key = f"{id_table} {id_col}"
    if new_key in ner_type:
        NER_type = ner_type[new_key]
        query = get_query(name, NER_type)
        match = re.search(r'Q(\d+)$', q_id)
        if match:
            queries.append((query, match[0]))

100%|██████████| 194438/194438 [00:02<00:00, 76779.83it/s] 


In [23]:
## QUERY CORRETTA CON FUZZYYYY

url = 'https://lamapi.hel.sintef.cloud/lookup/entity-retrieval'
sample_size = 4000
queries = random.sample(queries, sample_size)

# Backoff decorator for handling retries with exponential backoff
@backoff.on_exception(
    backoff.expo, 
    (aiohttp.ClientError, aiohttp.http_exceptions.HttpProcessingError, asyncio.TimeoutError), 
    max_tries=5, 
    max_time=300
)
async def fetch(session, url, params, headers, semaphore):
    async with semaphore:
        async with session.get(url, params=params, headers=headers, timeout=30) as response:
            try:
                response.raise_for_status()  # Raises an exception for 4XX/5XX status codes
                return await response.json()
            except Exception as e:
                return []
async def process_item(session, url, id, headers, params, semaphore, pbar):

    try:
        data = await fetch(session, url, params, headers, semaphore)
    except ClientResponseError as e:
        if e.status == 404:
            print(f"404 Error: Resource not found for '{name}'")
            pbar.update(1)  # No need to await here
            return 0, 0
        else:
            raise  # Re-raise the exception for other status codes

    num_result = len(data) if data else 0

    if data:
        for item in data:
            if id == item.get('id'):
                pbar.update(1)  # No need to await here
                pos_score = item.get('pos_score', 0)
                if pos_score:
                    mrr_increment = (num_result - (pos_score * num_result)) / num_result
                else:
                    mrr_increment = 1 / num_result  # Assume worst case for MRR if pos_score is 0
                return mrr_increment, 1

        
        #print(f"{name} NOT FOUND-->t{item}")

    return 0, 0

async def main(queries, url, pbar):
    headers = {'accept': 'application/json'}
    semaphore = asyncio.Semaphore(50)  # Limit to 50 concurrent requests
    m_mrr = 0
    cont_el = 0
    async with aiohttp.ClientSession() as session:
        tasks = []
        for param, id in queries:
            tasks.append(process_item(session, url, id, headers, param, semaphore, pbar))
        
        results = await asyncio.gather(*tasks)
        
        for (mrr_increment, count), (param, id) in zip(results, queries):
            if mrr_increment == 0 and count == 0:
                name = param['name']
                param['query'] = f'{{"query": {{"bool": {{"must": [{{"match": {{"name": {{"query": "{name}", "boost": 2.0, "fuzziness": "AUTO"}}}}}}]}}}}}}'

                response = requests.get(url, params=param)
                if response.status_code == 200:
                    data = response.json()
                    #print("after call")
                    num_result = len(data) if data else 0
                    if data:
                        for item in data:
                            if id == item.get('id'):
                                pbar.update(1)  # No need to await here
                                pos_score = item.get('pos_score', 0)
                                if pos_score:
                                    mrr_increment = (num_result - (pos_score * num_result)) / num_result
                                else:
                                    mrr_increment = 1 / num_result  # Assume worst case for MRR if pos_score is 0
                            
            m_mrr += mrr_increment
            cont_el += count

        pbar.close()  # No need to await here

    print(f"Coverage of R4: {cont_el / len(queries)}")
    print(f"Measure Reciprocal Rank of R4: {m_mrr / len(queries)}")

# Check if there's already a running event loop
if __name__ == "__main__":
    nest_asyncio.apply()  # Apply nest_asyncio
    try:
        pbar = tqdm(total=len(queries))
        asyncio.run(main(queries, url, pbar))
    except RuntimeError:  # For environments like Jupyter
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main(queries, url, pbar))


 67%|██████▋   | 2688/4000 [1:47:10<52:18,  2.39s/it]   


Coverage of R4: 0.48625
Measure Reciprocal Rank of R4: 0.5520435000000028


## Query
Coverage of R4: 0.48625

Measure Reciprocal Rank of R4: 0.55204350000000288

# Round4

In [24]:
####################
# READ THE JSON
#####################

json_file_path = "./data/Round4_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R4_sorted_mentions = json.load(file)

In [25]:
## Sample extraction
# SPLIT OVER THE QUARTILES

n = len(R4_sorted_mentions)
q1_idx = n // 4
q2_idx = n // 2
q3_idx = 3 * n // 4

# Step 3: Split the list into quartiles
q1 = R4_sorted_mentions[:q1_idx]
q2 = R4_sorted_mentions[q1_idx:q2_idx]
q3 = R4_sorted_mentions[q2_idx:q3_idx]
q4 = R4_sorted_mentions[q3_idx:]

sample_size = 1000
R4_sample_keys = []
R4_sample_keys = R4_sample_keys + random.sample(q1, sample_size)
R4_sample_keys = R4_sample_keys + random.sample(q2, sample_size)
R4_sample_keys = R4_sample_keys + random.sample(q3, sample_size)
R4_sample_keys = R4_sample_keys + random.sample(q4, sample_size)

q_ids = {item[1]['name']: item[1]['id'] for item in R4_sample_keys}

In [27]:
def get_wikidata_item_tree_item_idsSPARQL(root_items, forward_properties=None, backward_properties=None):
    """Return ids of WikiData items, which are in the tree spanned by the given root items and claims relating them
        to other items.

    :param root_items: iterable[int] One or multiple item entities that are the root elements of the tree
    :param forward_properties: iterable[int] | None property-claims to follow forward; that is, if root item R has
        a claim P:I, and P is in the list, the search will branch recursively to item I as well.
    :param backward_properties: iterable[int] | None property-claims to follow in reverse; that is, if (for a root
        item R) an item I has a claim P:R, and P is in the list, the search will branch recursively to item I as well.
    :return: iterable[int]: List with ids of WikiData items in the tree
    """

    query = '''PREFIX wikibase: <http://wikiba.se/ontology#>
            PREFIX wd: <http://www.wikidata.org/entity/>
            PREFIX wdt: <http://www.wikidata.org/prop/direct/>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>'''
    if forward_properties:
        query +='''SELECT ?WD_id WHERE {
                  ?tree0 (wdt:P%s)* ?WD_id .
                  BIND (wd:%s AS ?tree0)
                  }'''%( ','.join(map(str, forward_properties)),','.join(map(str, root_items)))
    elif backward_properties:
        query+='''SELECT ?WD_id WHERE {
                    ?WD_id (wdt:P%s)* wd:Q%s .
                    }'''%(','.join(map(str, backward_properties)), ','.join(map(str, root_items)))
    #print(query)

    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    data = get(url, params={'query': query, 'format': 'json'}).json()
    
    ids = []
    for item in data['results']['bindings']:
        this_id=item["WD_id"]["value"].split("/")[-1].lstrip("Q")
        #print(item)
        try:
            this_id = int(this_id)
            ids.append(this_id)
            #print(this_id)
        except ValueError:
            #print("exception")
            continue
    return ids


try:
    organization_subclass = get_wikidata_item_tree_item_idsSPARQL([43229], backward_properties=[279])
    #print(len(organization_subclass))
except json.decoder.JSONDecodeError:
    pass

try:
    country_subclass = get_wikidata_item_tree_item_idsSPARQL([6256], backward_properties=[279])
except json.decoder.JSONDecodeError:
    country_subclass = set()
    pass

try:
    city_subclass = get_wikidata_item_tree_item_idsSPARQL([515], backward_properties=[279])
except json.decoder.JSONDecodeError:
    city_subclass = set()
    pass

try:
    capitals_subclass = get_wikidata_item_tree_item_idsSPARQL([5119], backward_properties=[279])
except json.decoder.JSONDecodeError:
    capitals_subclass = set()
    pass

try:
    admTerr_subclass = get_wikidata_item_tree_item_idsSPARQL([15916867], backward_properties=[279])
except json.decoder.JSONDecodeError:
    admTerr_subclass = set()
    pass

try:
    family_subclass = get_wikidata_item_tree_item_idsSPARQL([17350442], backward_properties=[279])
except json.decoder.JSONDecodeError:
    family_subclass = set()
    pass

try:
    sportLeague_subclass = get_wikidata_item_tree_item_idsSPARQL([623109], backward_properties=[279])
except json.decoder.JSONDecodeError:
    sportLeague_subclass = set()
    pass

try:
    venue_subclass = get_wikidata_item_tree_item_idsSPARQL([8436], backward_properties=[279])
except json.decoder.JSONDecodeError:
    venue_subclass = set()
    pass
    
try:
    organization_subclass = list(set(organization_subclass) - set(country_subclass) - set(city_subclass) - set(capitals_subclass) - set(admTerr_subclass) - set(family_subclass) - set(sportLeague_subclass) - set(venue_subclass))
    #print(len(organization_subclass))
except json.decoder.JSONDecodeError:
    pass


try:
    geolocation_subclass = get_wikidata_item_tree_item_idsSPARQL([2221906], backward_properties=[279])
    #print(len(geolocation_subclass))
except json.decoder.JSONDecodeError:
    pass

try:
    food_subclass = get_wikidata_item_tree_item_idsSPARQL([2095], backward_properties=[279])
except json.decoder.JSONDecodeError:
    food_subclass = set()
    pass

try:
    edInst_subclass = get_wikidata_item_tree_item_idsSPARQL([2385804], backward_properties=[279])
except json.decoder.JSONDecodeError:
    edInst_subclass = set()
    pass

try:
    govAgency_subclass = get_wikidata_item_tree_item_idsSPARQL([327333], backward_properties=[279])
except json.decoder.JSONDecodeError:
    govAgency_subclass = set()
    pass

try:
    intOrg_subclass = get_wikidata_item_tree_item_idsSPARQL([484652], backward_properties=[279])
except json.decoder.JSONDecodeError:
    intOrg_subclass = set()
    pass

try:
    timeZone_subclass = get_wikidata_item_tree_item_idsSPARQL([12143], backward_properties=[279])
except json.decoder.JSONDecodeError:
    timeZone_subclass = set()
    pass
   
try:
    geolocation_subclass = list(set(geolocation_subclass) - set(food_subclass) - set(edInst_subclass) - set(govAgency_subclass) - set(intOrg_subclass) - set(timeZone_subclass))
    #print(len(geolocation_subclass))
except json.decoder.JSONDecodeError:
    pass

In [ ]:
tables = "./data/Dataset/Dataset/Round4_2020/tables/"
cta_file = './data/Dataset/Dataset/Round4_2020/gt/cta.csv'
os.listdir(tables)

def get_item_root(id_list):    
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    id_to_root_class = {}
    
    for el in tqdm(id_list, desc="Processing IDs"):
        if el not in id_to_root_class:
            query = f"""
            SELECT ?instanceClass ?instanceClassLabel WHERE {{
              wd:{el} wdt:P31 ?instanceClass .
              SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE]" }}
            }}
            """
            
            # Set the query and request JSON response
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)
            #time.sleep(0.5)
            
            try:
                results = sparql.query().convert()
                if len(results["results"]["bindings"]) > 0:
                    inst_item = int(results["results"]["bindings"][0]['instanceClassLabel']['value'][1:])
                    if inst_item in geolocation_subclass:
                        id_to_root_class[el] = "LOC"
                    elif inst_item in organization_subclass:
                        id_to_root_class[el] = "ORG"
                    elif inst_item == 5 or el == "Q5":
                        id_to_root_class[el] = "PERS"
                    else:
                        id_to_root_class[el] = "OTHERS"
                else:
                    id_to_root_class[el] = "OTHERS"
            except Exception as e:
                print(f"Error processing {el}: {e}")
                time.sleep(0.5)
                id_to_root_class[el] = None          
    
    return id_to_root_class

# Apply the function and create the 'key' column
root_classes = []
df = pd.read_csv(cta_file, header=None)
ids = [url.split('/')[-1] for url in df[2]]

root_classes = get_item_root(ids)

# Map root classes to categories
root_categories = []
for el in ids:
    root_categories.append(root_classes[el])


df["category"] = root_categories
cta_keys = {}
cta_keys["key"] = (df[0] + " " + df[1].astype('str'), df["category"])

key_to_cell = {}
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            key = f"{table_name} {col}"
            if key in set(cta_keys["key"][0].values):
                tmp_index = cta_keys["key"][0].values.tolist().index(key)
                tmp_value = cta_keys["key"][1].iloc[tmp_index]
                key_to_cell[key] = tmp_value
                #print(f"key: {key} -> key_to_cell[key]: {key_to_cell[key]}")

In [26]:
tables = "./data/Dataset/Dataset/Round4_2020/tables/"
cea_file = './data/Dataset/Dataset/Round4_2020/gt/cea.csv'
os.listdir(tables)
df = pd.read_csv(cea_file, header=None)
df["key"] = df[0] + " " + df[1].astype('str') + " " + df[2].astype('str')
cea_keys = (df["key"].values, df[3])
cea_values_dict = dict(zip(df["key"].values, df[3].values))
break
ner_type = {}
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            key = f"{table_name} {row+1} {col}"
            if key in cea_keys[0]:
                cell_value = df.iloc[row, col]
                ner_type[key] = (cell_value, cea_keys[1][cea_keys[0] == key])

SyntaxError: 'break' outside loop (1185396315.py, line 8)

In [ ]:
with open('./R4_key_to_cell.json', 'w') as json_file:
    json.dump(key_to_cell, json_file, indent=4)

In [27]:
import json

# Specify the path to the JSON file
file_path = "./R4_ner_type.json"

# Open and read the JSON file
with open(file_path, 'r') as f:
    ner_type = json.load(f)

with open('./R4_key_to_cell.json', 'r') as f:
    key_to_cell = json.load(f)

# Now key_to_cell contains the dictionary loaded from the JSON file
print("Dictionary loaded from JSON file:")


Dictionary loaded from JSON file:


In [28]:
def get_query(name, value):
    if value is not None:
         params = {
            'name': name,
            'token': 'lamapi_demo_2023',
            'kg': 'wikidata',
            'limit': 1000,
            'query': f'''
                {{
                    "query": {{
                        "bool": {{
                            "must": [
                                {{
                                    "match": {{
                                        "name": {{
                                            "query": "{name}",
                                            "boost": 2.0
                                        }}
                                    }}
                                }}
                            ],
                            "should": [
                                {{
                                    "term": {{
                                        "NERtype": "{value[1]}"
                                    }}
                                }}
                            ]
                        }}
                    }}
                }}
                ''',
            'sort': [
                f'''{{"popularity": {{"order": "desc"}}}}'''
            ]
        }
    else:
        params = {
            'name': name,
            'token': 'lamapi_demo_2023',
            'kg': 'wikidata',
            'limit': 1000,
            'query': f'{{"query": {{"bool": {{"must": [{{"match": {{"name": {{"query": "{name}", "boost": 2.0}}}}}}]}}}}}}',
            'sort': [
                f'''{{"popularity": {{"order": "desc"}}}}'''
            ]
        }
    return params


queries = []
for key in tqdm(key_to_cell):
    id_table, _, id_col = key.split(" ")
    name = key_to_cell[key]
    q_id = cea_values_dict[key]
    new_key = f"{id_table} {id_col}"
    if new_key in ner_type:
        NER_type = ner_type[new_key]
        query = get_query(name, NER_type)
        queries.append((query, re.search(r'Q(\d+)$', q_id)[0]))

100%|██████████| 994921/994921 [00:29<00:00, 33797.28it/s] 


In [29]:
## QUERY CORRETTA CON FUZZYYYY

url = 'https://lamapi.hel.sintef.cloud/lookup/entity-retrieval'
sample_size = 4000
queries = random.sample(queries, sample_size)

# Backoff decorator for handling retries with exponential backoff
@backoff.on_exception(
    backoff.expo, 
    (aiohttp.ClientError, aiohttp.http_exceptions.HttpProcessingError, asyncio.TimeoutError), 
    max_tries=5, 
    max_time=300
)
async def fetch(session, url, params, headers, semaphore):
    async with semaphore:
        async with session.get(url, params=params, headers=headers, timeout=30) as response:
            try:
                response.raise_for_status()  # Raises an exception for 4XX/5XX status codes
                return await response.json()
            except Exception as e:
                return []
async def process_item(session, url, id, headers, params, semaphore, pbar):

    try:
        data = await fetch(session, url, params, headers, semaphore)
    except ClientResponseError as e:
        if e.status == 404:
            print(f"404 Error: Resource not found for '{name}'")
            pbar.update(1)  # No need to await here
            return 0, 0
        else:
            raise  # Re-raise the exception for other status codes

    num_result = len(data) if data else 0

    if data:
        for item in data:
            if id == item.get('id'):
                pbar.update(1)  # No need to await here
                pos_score = item.get('pos_score', 0)
                if pos_score:
                    mrr_increment = (num_result - (pos_score * num_result)) / num_result
                else:
                    mrr_increment = 1 / num_result  # Assume worst case for MRR if pos_score is 0
                return mrr_increment, 1

        
        #print(f"{name} NOT FOUND-->t{item}")

    return 0, 0

async def main(queries, url, pbar):
    headers = {'accept': 'application/json'}
    semaphore = asyncio.Semaphore(50)  # Limit to 50 concurrent requests
    m_mrr = 0
    cont_el = 0
    async with aiohttp.ClientSession() as session:
        tasks = []
        for param, id in queries:
            tasks.append(process_item(session, url, id, headers, param, semaphore, pbar))
        
        results = await asyncio.gather(*tasks)
        
        for (mrr_increment, count), (param, id) in zip(results, queries):
            if mrr_increment == 0 and count == 0:
                name = param['name']
                param['query'] = f'{{"query": {{"bool": {{"must": [{{"match": {{"name": {{"query": "{name}", "boost": 2.0, "fuzziness": "AUTO"}}}}}}]}}}}}}'

                response = requests.get(url, params=param)
                if response.status_code == 200:
                    data = response.json()
                    #print("after call")
                    num_result = len(data) if data else 0
                    if data:
                        for item in data:
                            if id == item.get('id'):
                                pbar.update(1)  # No need to await here
                                pos_score = item.get('pos_score', 0)
                                if pos_score:
                                    mrr_increment = (num_result - (pos_score * num_result)) / num_result
                                else:
                                    mrr_increment = 1 / num_result  # Assume worst case for MRR if pos_score is 0
                            
            m_mrr += mrr_increment
            cont_el += count

        pbar.close()  # No need to await here

    print(f"Coverage of R4: {cont_el / len(queries)}")
    print(f"Measure Reciprocal Rank of R4: {m_mrr / len(queries)}")

# Check if there's already a running event loop
if __name__ == "__main__":
    nest_asyncio.apply()  # Apply nest_asyncio
    try:
        pbar = tqdm(total=len(queries))
        asyncio.run(main(queries, url, pbar))
    except RuntimeError:  # For environments like Jupyter
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main(queries, url, pbar))


 98%|█████████▊| 3924/4000 [17:31<00:20,  3.73it/s]


Coverage of R4: 0.94025
Measure Reciprocal Rank of R4: 0.915011999999962


## query
Coverage of R4: 0.94025

Measure Reciprocal Rank of R4: 0.9150119999999621

# HardTableR1

In [3]:
####################
# READ THE JSON
#####################

json_file_path = "./data/HardTablesR2_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    HT2_sorted_mentions = json.load(file)

In [4]:
# SPLIT OVER THE QUARTILES

n = len(HT2_sorted_mentions)
q1_idx = n // 4
q2_idx = n // 2
q3_idx = 3 * n // 4

# Step 3: Split the list into quartiles
q1 = HT2_sorted_mentions[:q1_idx]
q2 = HT2_sorted_mentions[q1_idx:q2_idx]
q3 = HT2_sorted_mentions[q2_idx:q3_idx]
q4 = HT2_sorted_mentions[q3_idx:]

sample_size = 1000
HT2_sample_keys = []
HT2_sample_keys = HT2_sample_keys + random.sample(q1, sample_size)
HT2_sample_keys = HT2_sample_keys + random.sample(q2, sample_size)
HT2_sample_keys = HT2_sample_keys + random.sample(q3, sample_size)
HT2_sample_keys = HT2_sample_keys + random.sample(q4, sample_size)

q_ids = {item[1]['name']: item[1]['id'] for item in HT2_sample_keys}

In [ ]:
def get_wikidata_item_tree_item_idsSPARQL(root_items, forward_properties=None, backward_properties=None):
    """Return ids of WikiData items, which are in the tree spanned by the given root items and claims relating them
        to other items.

    :param root_items: iterable[int] One or multiple item entities that are the root elements of the tree
    :param forward_properties: iterable[int] | None property-claims to follow forward; that is, if root item R has
        a claim P:I, and P is in the list, the search will branch recursively to item I as well.
    :param backward_properties: iterable[int] | None property-claims to follow in reverse; that is, if (for a root
        item R) an item I has a claim P:R, and P is in the list, the search will branch recursively to item I as well.
    :return: iterable[int]: List with ids of WikiData items in the tree
    """

    query = '''PREFIX wikibase: <http://wikiba.se/ontology#>
            PREFIX wd: <http://www.wikidata.org/entity/>
            PREFIX wdt: <http://www.wikidata.org/prop/direct/>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>'''
    if forward_properties:
        query +='''SELECT ?WD_id WHERE {
                  ?tree0 (wdt:P%s)* ?WD_id .
                  BIND (wd:%s AS ?tree0)
                  }'''%( ','.join(map(str, forward_properties)),','.join(map(str, root_items)))
    elif backward_properties:
        query+='''SELECT ?WD_id WHERE {
                    ?WD_id (wdt:P%s)* wd:Q%s .
                    }'''%(','.join(map(str, backward_properties)), ','.join(map(str, root_items)))
    #print(query)

    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    data = get(url, params={'query': query, 'format': 'json'}).json()
    
    ids = []
    for item in data['results']['bindings']:
        this_id=item["WD_id"]["value"].split("/")[-1].lstrip("Q")
        #print(item)
        try:
            this_id = int(this_id)
            ids.append(this_id)
            #print(this_id)
        except ValueError:
            #print("exception")
            continue
    return ids


try:
    organization_subclass = get_wikidata_item_tree_item_idsSPARQL([43229], backward_properties=[279])
    #print(len(organization_subclass))
except json.decoder.JSONDecodeError:
    pass

try:
    country_subclass = get_wikidata_item_tree_item_idsSPARQL([6256], backward_properties=[279])
except json.decoder.JSONDecodeError:
    country_subclass = set()
    pass

try:
    city_subclass = get_wikidata_item_tree_item_idsSPARQL([515], backward_properties=[279])
except json.decoder.JSONDecodeError:
    city_subclass = set()
    pass

try:
    capitals_subclass = get_wikidata_item_tree_item_idsSPARQL([5119], backward_properties=[279])
except json.decoder.JSONDecodeError:
    capitals_subclass = set()
    pass

try:
    admTerr_subclass = get_wikidata_item_tree_item_idsSPARQL([15916867], backward_properties=[279])
except json.decoder.JSONDecodeError:
    admTerr_subclass = set()
    pass

try:
    family_subclass = get_wikidata_item_tree_item_idsSPARQL([17350442], backward_properties=[279])
except json.decoder.JSONDecodeError:
    family_subclass = set()
    pass

try:
    sportLeague_subclass = get_wikidata_item_tree_item_idsSPARQL([623109], backward_properties=[279])
except json.decoder.JSONDecodeError:
    sportLeague_subclass = set()
    pass

try:
    venue_subclass = get_wikidata_item_tree_item_idsSPARQL([8436], backward_properties=[279])
except json.decoder.JSONDecodeError:
    venue_subclass = set()
    pass
    
try:
    organization_subclass = list(set(organization_subclass) - set(country_subclass) - set(city_subclass) - set(capitals_subclass) - set(admTerr_subclass) - set(family_subclass) - set(sportLeague_subclass) - set(venue_subclass))
    #print(len(organization_subclass))
except json.decoder.JSONDecodeError:
    pass


try:
    geolocation_subclass = get_wikidata_item_tree_item_idsSPARQL([2221906], backward_properties=[279])
    #print(len(geolocation_subclass))
except json.decoder.JSONDecodeError:
    pass

try:
    food_subclass = get_wikidata_item_tree_item_idsSPARQL([2095], backward_properties=[279])
except json.decoder.JSONDecodeError:
    food_subclass = set()
    pass

try:
    edInst_subclass = get_wikidata_item_tree_item_idsSPARQL([2385804], backward_properties=[279])
except json.decoder.JSONDecodeError:
    edInst_subclass = set()
    pass

try:
    govAgency_subclass = get_wikidata_item_tree_item_idsSPARQL([327333], backward_properties=[279])
except json.decoder.JSONDecodeError:
    govAgency_subclass = set()
    pass

try:
    intOrg_subclass = get_wikidata_item_tree_item_idsSPARQL([484652], backward_properties=[279])
except json.decoder.JSONDecodeError:
    intOrg_subclass = set()
    pass

try:
    timeZone_subclass = get_wikidata_item_tree_item_idsSPARQL([12143], backward_properties=[279])
except json.decoder.JSONDecodeError:
    timeZone_subclass = set()
    pass
   
try:
    geolocation_subclass = list(set(geolocation_subclass) - set(food_subclass) - set(edInst_subclass) - set(govAgency_subclass) - set(intOrg_subclass) - set(timeZone_subclass))
    #print(len(geolocation_subclass))
except json.decoder.JSONDecodeError:
    pass

try:
    human_subclass = get_wikidata_item_tree_item_idsSPARQL([5], backward_properties=[279])
    print(human_subclass)
except json.decoder.JSONDecodeError:
    human_subclass = set()
    pass

In [10]:

tables_path = "./data/Dataset/Dataset/HardTablesR2/tables/"
cea_file = './data/Dataset/Dataset/HardTablesR2/gt/cea.csv'
os.listdir(tables_path)
# Initialize logging
#logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Read the cea_file and create a key-value dictionary
df = pd.read_csv(cea_file, header=None)
df["key"] = df[0] + " " + df[1].astype(str) + " " + df[2].astype(str)
cea_values_dict = dict(zip(df["key"].values, df[3].values))
cea_keys_set = set(df["key"].values)

# Function to process a single table file
def process_table_file(table_file):
    try:
        table_name = os.path.splitext(os.path.basename(table_file))[0]
        df = pd.read_csv(table_file)
        local_key_to_cell = {}
        
        for row in range(df.shape[0]):
            for col in range(df.shape[1]):
                key = f"{table_name} {row+1} {col}"
                if key in cea_keys_set:
                    cell_value = df.iloc[row, col]
                    local_key_to_cell[key] = (cell_value, cea_values_dict[key])
                    break  # Exit inner loop early as only one match per row/col is needed
        
        return local_key_to_cell
    except Exception as e:
        logging.error(f"Error processing {table_file}: {e}")
        return {}

pattern = r'^\.'

# Create a list of file paths, excluding files that start with a dot
table_files = [os.path.join(tables_path, table) for table in os.listdir(tables_path) if not re.match(pattern, table)]

# Process tables sequentially
key_to_cell = {}
for table_file in tqdm(table_files, desc="Processing tables"):
    local_key_to_cell = process_table_file(table_file)
    key_to_cell.update(local_key_to_cell)

Processing tables: 100%|██████████| 1750/1750 [00:35<00:00, 48.73it/s]


In [11]:
tables = "./data/Dataset/Dataset/HardTablesR2/tables/"
cea_file = './data/Dataset/Dataset/HardTablesR2/gt/cea.csv'
cta_file = './data/Dataset/Dataset/HardTablesR2/gt/cta.csv'
os.listdir(tables)

def get_item_root(id_list):     
    id_to_root_class = {}
    for el in id_list:
        inst_item = int(re.search(r'(\d+)$', el)[0])
        if inst_item in geolocation_subclass:
            #id_to_root_class[el] = "LOC"
            return "LOC"
        elif inst_item in organization_subclass:
            #id_to_root_class[el] = "ORG"
            return "ORG"
        elif inst_item in human_subclass:
            #id_to_root_class[el] = "PERS"
            return "PERS"      
    
    return "OTHERS"

# Apply the function and create the 'key' column
root_classes = []
df = pd.read_csv(cta_file, header=None)
root_categories = []
for urls in df[2]:
    tmp = [url.split('/')[-1] for url in urls.split(" ")]
    root_categories.append(get_item_root(tmp))




df["category"] = root_categories
cta_keys = {}
cta_keys["key"] = (df[0] + " " + df[1].astype('str'), df["category"])

ner_type = {}
for table in tqdm(os.listdir(tables)):
    pattern = r'^\.'
    if re.match(pattern, table):
        continue
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            key = f"{table_name} {col}"
            if key in set(cta_keys["key"][0].values):
                tmp_index = cta_keys["key"][0].values.tolist().index(key)
                tmp_value = cta_keys["key"][1].iloc[tmp_index]
                ner_type[key] = tmp_value

100%|██████████| 2692/2692 [00:47<00:00, 56.80it/s] 


In [55]:
def convert_int64_to_int(obj):
    if isinstance(obj, dict):
        return {k: convert_int64_to_int(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_int64_to_int(i) for i in obj]
    elif isinstance(obj, np.int64):
        return int(obj)
    else:
        return obj

key_to_cell = convert_int64_to_int(key_to_cell)

In [61]:
# Convert non-serializable values to serializable ones
key_to_cell_serializable = convert_to_serializable(key_to_cell)

# Write to JSON files
with open('./HT2_key_to_cell.json', 'w') as json_file:
    json.dump(key_to_cell_serializable, json_file, indent=4)

# Assuming ner_type is another dictionary you need to serialize
ner_type = key_to_cell_serializable  # Replace with your actual ner_type dictionary
with open('./HT2_ner_type.json', 'w') as json_file:
    json.dump(ner_type, json_file, indent=4)

NameError: name 'convert_to_serializable' is not defined

In [56]:
with open('./HT2_key_to_cell.json', 'w') as json_file:
    json.dump(key_to_cell, json_file, indent=4)

with open('./HT2_ner_type.json', 'w') as json_file:
    json.dump(ner_type, json_file, indent=4)

TypeError: Object of type int64 is not JSON serializable

In [2]:

file_path = "./HT2_ner_type.json"

# Open and read the JSON file
with open(file_path, 'r') as f:
    ner_type = json.load(f)

with open('./HT2_key_to_cell.json', 'r') as f:
    key_to_cell = json.load(f)

# Now key_to_cell contains the dictionary loaded from the JSON file
print("Dictionary loaded from JSON file:")


JSONDecodeError: Expecting value: line 74891 column 9 (char 2092575)

In [12]:
def get_query(name, value):
    if value is not None:
         params = {
            'name': name,
            'token': 'lamapi_demo_2023',
            'kg': 'wikidata',
            'limit': 1000,
            'query': f'''
                {{
                    "query": {{
                        "bool": {{
                            "should": [
                                {{
                                    "match": {{
                                        "name": {{
                                            "query": "{name}",
                                            "boost": 2.0
                                        }}
                                    }}
                                }}
                            ],
                            "should": [
                                {{
                                    "term": {{
                                        "NERtype": "{value[1]}"
                                    }}
                                }}
                            ]
                        }}
                    }}
                }}
                ''',
            'sort': [
                f'''{{"popularity": {{"order": "desc"}}}}'''
            ]
        }
    else:
        params = {
            'name': name,
            'token': 'lamapi_demo_2023',
            'kg': 'wikidata',
            'limit': 1000,
            'query': f'{{"query": {{"bool": {{"must": [{{"match": {{"name": {{"query": "{name}", "boost": 2.0}}}}}}]}}}}}}',
            'sort': [
                f'''{{"popularity": {{"order": "desc"}}}}'''
            ]
        }
    return params


queries = []
for key in tqdm(key_to_cell):
    id_table, _, id_col = key.split(" ")
    name = key_to_cell[key][0]
    q_id = key_to_cell[key][1]
    new_key = f"{id_table} {id_col}"
    if new_key in ner_type:
        NER_type = ner_type[new_key]
        query = get_query(name, NER_type)
        match = re.search(r'Q(\d+)$', q_id)
        if match:
            queries.append((query, match[0]))

100%|██████████| 29280/29280 [00:03<00:00, 9218.70it/s] 


In [13]:
## QUERY CORRETTA CON FUZZYYYY

url = 'https://lamapi.hel.sintef.cloud/lookup/entity-retrieval'
sample_size = 4000
queries = random.sample(queries, sample_size)

# Backoff decorator for handling retries with exponential backoff
@backoff.on_exception(
    backoff.expo, 
    (aiohttp.ClientError, aiohttp.http_exceptions.HttpProcessingError, asyncio.TimeoutError), 
    max_tries=5, 
    max_time=300
)
async def fetch(session, url, params, headers, semaphore):
    async with semaphore:
        async with session.get(url, params=params, headers=headers, timeout=30) as response:
            try:
                response.raise_for_status()  # Raises an exception for 4XX/5XX status codes
                return await response.json()
            except Exception as e:
                return []
async def process_item(session, url, id, headers, params, semaphore, pbar):

    try:
        data = await fetch(session, url, params, headers, semaphore)
    except ClientResponseError as e:
        if e.status == 404:
            print(f"404 Error: Resource not found for '{name}'")
            pbar.update(1)  # No need to await here
            return 0, 0
        else:
            raise  # Re-raise the exception for other status codes

    num_result = len(data) if data else 0

    if data:
        for item in data:
            if id == item.get('id'):
                pbar.update(1)  # No need to await here
                pos_score = item.get('pos_score', 0)
                if pos_score:
                    mrr_increment = (num_result - (pos_score * num_result)) / num_result
                else:
                    mrr_increment = 1 / num_result  # Assume worst case for MRR if pos_score is 0
                return mrr_increment, 1

        
        #print(f"{name} NOT FOUND-->t{item}")

    return 0, 0

async def main(queries, url, pbar):
    headers = {'accept': 'application/json'}
    semaphore = asyncio.Semaphore(50)  # Limit to 50 concurrent requests
    m_mrr = 0
    cont_el = 0
    async with aiohttp.ClientSession() as session:
        tasks = []
        for param, id in queries:
            tasks.append(process_item(session, url, id, headers, param, semaphore, pbar))
        
        results = await asyncio.gather(*tasks)
        print("fuzzy")
        
        for (mrr_increment, count), (param, id) in zip(results, queries):
            if mrr_increment == 0 and count == 0:
                name = param['name']
                param['query'] = f'{{"query": {{"bool": {{"should": [{{"match": {{"name": {{"query": "{name}", "boost": 2.0, "fuzziness": "AUTO"}}}}}}]}}}}}}'

                response = requests.get(url, params=param)
                if response.status_code == 200:
                    data = response.json()
                    #print("after call")
                    num_result = len(data) if data else 0
                    if data:
                        for item in data:
                            if id == item.get('id'):
                                pbar.update(1)  # No need to await here
                                pos_score = item.get('pos_score', 0)
                                if pos_score:
                                    mrr_increment = (num_result - (pos_score * num_result)) / num_result
                                else:
                                    mrr_increment = 1 / num_result  # Assume worst case for MRR if pos_score is 0
                            
            m_mrr += mrr_increment
            cont_el += count

        pbar.close()  # No need to await here

    print(f"Coverage of R4: {cont_el / len(queries)}")
    print(f"Measure Reciprocal Rank of R4: {m_mrr / len(queries)}")

# Check if there's already a running event loop
if __name__ == "__main__":
    nest_asyncio.apply()  # Apply nest_asyncio
    try:
        pbar = tqdm(total=len(queries))
        asyncio.run(main(queries, url, pbar))
    except RuntimeError:  # For environments like Jupyter
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main(queries, url, pbar))


  0%|          | 0/4000 [00:00<?, ?it/s]

NameError: name 'ClientResponseError' is not defined